<a href="https://colab.research.google.com/github/manushi0304/Diabetic_Retinopathy/blob/main/DenseNetQuantized_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# Robust AIT (Average Inference Time) for ONE quantized TFLite model on 10 images
# - If test.csv has bad paths, it FALLS BACK to Kaggle APTOS images automatically.
# - Works for FP16/INT8/FP32 TFLite.
# ============================================

from google.colab import drive
drive.mount('/content/drive')

import os, time, glob, numpy as np, pandas as pd
from PIL import Image
from tqdm import tqdm
import tensorflow as tf

# ========= USER CONFIG =========
MODEL_PATH = "/content/drive/MyDrive/DiabeticProject/tflite/DenseNet121_model_fp16.tflite"  # <-- change if needed
LIMIT_IMAGES = 10
WARMUP_RUNS = 5
NUM_THREADS = 4
FALLBACK_KAGGLE_DATASET = "subhajeetdas/aptos-2019-jpg"  # used only if test.csv is missing/broken
TEST_CSV = "/content/drive/MyDrive/DiabeticProject/test.csv"
# ==============================

def file_mb(path):
    try: return round(os.path.getsize(path)/(1024*1024), 3)
    except: return None

def get_preprocess_from_name(model_path):
    name = os.path.basename(model_path).lower()
    f = lambda x: x / 255.0
    if "efficientnetv2" in name or "efficientnetv2b0" in name:
        f = tf.keras.applications.efficientnet_v2.preprocess_input
    elif "inceptionv3" in name:
        f = tf.keras.applications.inception_v3.preprocess_input
    elif "mobilenetv2" in name:
        f = tf.keras.applications.mobilenet_v2.preprocess_input
    elif "resnet50v2" in name or "resnetv2" in name:
        f = tf.keras.applications.resnet_v2.preprocess_input
    elif "nasnetmobile" in name or "nasnet" in name:
        f = tf.keras.applications.nasnet.preprocess_input
    elif "densenet121" in name or "densenet" in name:
        f = tf.keras.applications.densenet.preprocess_input
    return f

def quantize_np(x_float, scale, zp, dtype):
    if scale is None or scale == 0:
        return x_float.astype(dtype)
    q = np.round(x_float / scale + zp)
    if np.issubdtype(dtype, np.integer):
        info = np.iinfo(dtype); q = np.clip(q, info.min, info.max)
    return q.astype(dtype)

def load_and_prep(path, target_hw, preprocess_fn):
    img = Image.open(path).convert("RGB").resize(target_hw, Image.BILINEAR)
    x = np.array(img, dtype=np.float32)
    x = preprocess_fn(x)
    return np.expand_dims(x, 0)

def pick_10_image_paths(test_csv_path, fallback_dataset):
    # 1) Try test.csv
    if os.path.exists(test_csv_path):
        try:
            df = pd.read_csv(test_csv_path)
            if "filepath" in df.columns:
                df = df[df["filepath"].apply(os.path.exists)]
                if len(df) >= 1:
                    return df.sample(n=min(LIMIT_IMAGES, len(df)), random_state=42)["filepath"].tolist()
        except Exception:
            pass  # fall through to kagglehub
    # 2) Fallback: Kaggle APTOS dataset
    try:
        import kagglehub
        dpath = kagglehub.dataset_download(fallback_dataset)
        imgs = sorted(glob.glob(os.path.join(dpath, "**", "*.png"), recursive=True))
        if not imgs:
            imgs = sorted(glob.glob(os.path.join(dpath, "**", "*.jpg"), recursive=True))
        if not imgs:
            raise FileNotFoundError("No images found in Kaggle fallback.")
        rng = np.random.default_rng(42)
        idx = rng.choice(len(imgs), size=min(LIMIT_IMAGES, len(imgs)), replace=False)
        return [imgs[i] for i in idx]
    except Exception as e:
        raise FileNotFoundError(
            "No usable images. Fix paths in test.csv or ensure Kaggle dataset can be downloaded.\n"
            f"Original error: {e}"
        )

def average_inference_time_tflite(model_path, image_paths):
    interpreter = tf.lite.Interpreter(model_path=model_path, num_threads=NUM_THREADS)
    interpreter.allocate_tensors()
    in_det = interpreter.get_input_details()[0]
    out_det = interpreter.get_output_details()[0]

    _, H, W, C = in_det["shape"]
    in_dtype = in_det["dtype"]
    in_scale, in_zp = in_det.get("quantization", (None, None))

    preprocess_fn = get_preprocess_from_name(model_path)

    # Warm-up
    dummy = np.zeros((1, H, W, C), dtype=np.float32)
    dummy = preprocess_fn(dummy)
    dummy_q = quantize_np(dummy, in_scale, in_zp, in_dtype) if in_dtype != np.float32 else dummy.astype(in_dtype)
    interpreter.set_tensor(in_det["index"], dummy_q)
    for _ in range(max(0, WARMUP_RUNS)):
        interpreter.invoke(); _ = interpreter.get_tensor(out_det["index"])

    # Timing
    times = []
    for p in tqdm(image_paths, desc=os.path.basename(model_path)):
        x = load_and_prep(p, (W, H), preprocess_fn)
        x = quantize_np(x, in_scale, in_zp, in_dtype) if in_dtype != np.float32 else x.astype(in_dtype)
        interpreter.set_tensor(in_det["index"], x)
        t0 = time.perf_counter(); interpreter.invoke(); t1 = time.perf_counter()
        _ = interpreter.get_tensor(out_det["index"])
        times.append((t1 - t0) * 1000.0)

    times = np.array(times, dtype=np.float64)
    return {
        "model_path": model_path,
        "images": len(image_paths),
        "input_shape": [int(d) for d in in_det["shape"]],
        "input_dtype": str(in_dtype),
        "tflite_size_mb": file_mb(model_path),
        "avg_ms": float(times.mean()),
        "median_ms": float(np.median(times)),
        "p95_ms": float(np.percentile(times, 95)),
        "min_ms": float(times.min()),
        "max_ms": float(times.max()),
        "fps": float(1000.0 / times.mean()) if times.mean() > 0 else None,
    }

# ---------- run ----------
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"TFLite model not found: {MODEL_PATH}")

image_paths = pick_10_image_paths(TEST_CSV, FALLBACK_KAGGLE_DATASET)
print("Images to evaluate:")
for p in image_paths: print(" -", p)

res = average_inference_time_tflite(MODEL_PATH, image_paths)

print("\n===== Average Inference Time (Quantized TFLite) =====")
print(f"Model              : {os.path.basename(res['model_path'])}")
print(f"Model size (MB)    : {res['tflite_size_mb']}")
print(f"Input shape/dtype  : {res['input_shape']} / {res['input_dtype']}")
print(f"Images evaluated   : {res['images']}")
print(f"Avg per image (ms) : {res['avg_ms']:.3f}")
print(f"Median (ms)        : {res['median_ms']:.3f}")
print(f"P95 (ms)           : {res['p95_ms']:.3f}")
print(f"Min/Max (ms)       : {res['min_ms']:.3f} / {res['max_ms']:.3f}")
print(f"Approx FPS         : {res['fps']:.2f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


100%|██████████| 2.82G/2.82G [00:26<00:00, 115MB/s]

Extracting files...


Images to evaluate:
 - /root/.cache/kagglehub/datasets/subhajeetdas/aptos-2019-jpg/versions/12/APTOS 2019 (Original) (Binary)/DR/image_00630.png
 - /root/.cache/kagglehub/datasets/subhajeetdas/aptos-2019-jpg/versions/12/APTOS 2019 (Original) (Binary)/No DR/image_01569.png
 - /root/.cache/kagglehub/datasets/subhajeetdas/aptos-2019-jpg/versions/12/APTOS 2019 (Original) (Binary)/DR/image_00655.png
 - /root/.cache/kagglehub/datasets/subhajeetdas/aptos-2019-jpg/versions/12/APTOS 2019 (Original) (Binary)/No DR/image_00712.png
 - /root/.cache/kagglehub/datasets/subhajeetdas/aptos-2019-jpg/versions/12/APTOS 2019 (Original) (Binary)/DR/image_3 (143).png
 - /root/.cache/kagglehub/datasets/subhajeetdas/aptos-2019-jpg/versions/12/APTOS 2019 (Original) (Binary)/DR/image_3 (124).png
 - /root/.cache/kagglehub/datasets/subhajeetdas/aptos-2019-jpg/versions/12/APTOS 2019 (Original) (Binary)/No DR/image_01004.png
 - /root/.cache/kagglehub/datasets/subhajeetdas/aptos-2019-jpg/versions/12/APTOS 2019 (Origi

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
DenseNet121_model_fp16.tflite: 100%|██████████| 10/10 [00:22<00:00,  2.26s/it]


===== Average Inference Time (Quantized TFLite) =====
Model              : DenseNet121_model_fp16.tflite
Model size (MB)    : 14.855
Input shape/dtype  : [1, 256, 256, 3] / <class 'numpy.float32'>
Images evaluated   : 10
Avg per image (ms) : 2181.291
Median (ms)        : 1933.780
P95 (ms)           : 3336.351
Min/Max (ms)       : 1757.277 / 3597.289
Approx FPS         : 0.46
